In [3]:
import pandas as pd
import numpy as np
import os
import lightgbm as lgb
import gc
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge

print("--- FINAL ULTRA-FAST SCRIPT (CORRECTED) ---")

# --- STAGE 1: LOAD FEATURES ---
PROCESSED_DIR = "../data/processed"
full_df_text_stats = pd.read_parquet(os.path.join(PROCESSED_DIR, "features_v1.parquet"))
text_embeddings_df = pd.read_parquet(os.path.join(PROCESSED_DIR, "text_embeddings_v1.parquet"))
full_df = pd.concat([full_df_text_stats.reset_index(drop=True), text_embeddings_df.reset_index(drop=True)], axis=1)

# --- STAGE 2: DATA PREPARATION ---
train_df = full_df[full_df['is_train'] == 1].copy()
test_df = full_df[full_df['is_train'] == 0].copy()

y = train_df['log_price']
y_true_prices = train_df['price']

features_to_drop = ['sample_id', 'price', 'log_price', 'is_train']
features = [col for col in full_df.columns if col not in features_to_drop]

# Convert categorical features BEFORE creating X and X_test
categorical_features = ['extracted_unit', 'brand']
for col in categorical_features:
    full_df[col] = full_df[col].astype('category')

X = full_df[full_df['is_train'] == 1][features]
X_test = full_df[full_df['is_train'] == 0][features]

del full_df, train_df, test_df, full_df_text_stats, text_embeddings_df
gc.collect()
print("Data preparation complete.")

# --- STAGE 3: TRAINING ---
N_SPLITS = 5
kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=42)
oof_predictions_log = np.zeros(len(X))
test_predictions_log = np.zeros(len(X_test))

print("--- Starting ULTRA-FAST training ---")
for fold, (train_index, val_index) in enumerate(kf.split(X, y)):
    print(f"--- Fold {fold+1}/{N_SPLITS} ---")
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]
    
    lgb_params = { 'objective': 'regression_l1', 'metric': 'rmse', 'n_estimators': 2500,
                   'learning_rate': 0.04, 'feature_fraction': 0.8, 'bagging_fraction': 0.8, 'bagging_freq': 1,
                   'lambda_l1': 0.1, 'lambda_l2': 0.1, 'num_leaves': 31, 'verbose': -1, 'n_jobs': -1, 'seed': 42 + fold }
    
    model = lgb.LGBMRegressor(**lgb_params)
    def lgbm_smape(y_true_log, y_pred_log):
        y_true = np.expm1(y_true_log); y_pred = np.expm1(y_pred_log)
        denom = np.abs(y_true) + np.abs(y_pred)
        return 'SMAPE', 100.0 * np.mean(2 * np.abs(y_true - y_pred) / np.clip(denom, 1e-8, None)), False
    
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], eval_metric=lgbm_smape, 
              categorical_feature='auto', callbacks=[lgb.early_stopping(100, verbose=False)])
    
    oof_predictions_log[val_index] = model.predict(X_val)
    test_predictions_log += model.predict(X_test) / N_SPLITS

# --- STAGE 4: POST-PROCESSING AND SUBMISSION ---
print("\n--- Training complete. Starting post-processing. ---")
# ... (The rest of the script is the same post-processing and submission code)
y_true = y_true_prices.values
train_brands = X['brand']; test_brands = X_test['brand']

def smape(y, y_pred):
    denom = np.abs(y) + np.abs(y_pred)
    return 100.0 * np.mean(2 * np.abs(y - y_pred) / np.clip(denom, 1e-8, None))

oof_preds = np.expm1(oof_predictions_log); test_preds = np.expm1(test_predictions_log)
print(f"Initial OOF SMAPE: {smape(y_true, oof_preds):.4f}")

low, high = np.percentile(y_true, [0.5, 99.5])
oof_preds = np.clip(oof_preds, low, high); test_preds = np.clip(test_preds, low, high)
print(f"After Winsorizing: {smape(y_true, oof_preds):.4f}")

oof_df = pd.DataFrame({'brand': train_brands, 'y_true': y_true, 'pred': oof_preds})
group_shift = oof_df.groupby('brand').apply(lambda g: g['y_true'].mean() - g['pred'].mean()).to_dict()
oof_preds += train_brands.map(group_shift).fillna(0.0).values
test_preds += test_brands.map(group_shift).fillna(0.0).values
print(f"After Group Correction: {smape(y_true, oof_preds):.4f}")

meta_model = Ridge(alpha=1.0).fit(oof_preds.reshape(-1, 1), y_true)
oof_preds = meta_model.predict(oof_preds.reshape(-1, 1)); test_preds = meta_model.predict(test_preds.reshape(-1, 1))
print(f"After Final Calibration: {smape(y_true, oof_preds):.4f}")

final_test_preds = np.clip(test_preds, 0.01, None)
print(f"\nFINAL OOF SMAPE SCORE: {smape(y_true, oof_preds):.4f}")

original_test_df = pd.read_csv("../data/raw/dataset/test.csv")
submission_df = pd.DataFrame({'sample_id': original_test_df['sample_id'], 'price': final_test_preds})
submission_df.to_csv("../test_out.csv", index=False)
print(f"\n✅✅✅ FINAL SUBMISSION FILE CREATED: 'test_out.csv' ✅✅✅")

--- FINAL ULTRA-FAST SCRIPT (CORRECTED) ---
Data preparation complete.
--- Starting ULTRA-FAST training ---
--- Fold 1/5 ---
--- Fold 2/5 ---
--- Fold 3/5 ---
--- Fold 4/5 ---
--- Fold 5/5 ---

--- Training complete. Starting post-processing. ---
Initial OOF SMAPE: 54.1383
After Winsorizing: 54.1379
After Group Correction: 22.9088
After Final Calibration: 24.3318

FINAL OOF SMAPE SCORE: 24.3318

✅✅✅ FINAL SUBMISSION FILE CREATED: 'test_out.csv' ✅✅✅


In [4]:
import numpy as np
import pandas as pd
from scipy.stats import rankdata
from tqdm import tqdm

print("--- STARTING AT-ANY-COST HYPER-AGGRESSIVE POST-PROCESSING ---")

# --- We will use the variables from the previous cell:
# y_true_prices, oof_predictions_log, test_predictions_log, X, X_test

# --- Define the canonical SMAPE function ---
def smape(y_true, y_pred):
    denom = np.abs(y_true) + np.abs(y_pred)
    return 100.0 * np.mean(2 * np.abs(y_true - y_pred) / np.clip(denom, 1e-8, None))

# --- Invert from log scale ---
oof_preds = np.expm1(oof_predictions_log)
test_preds = np.expm1(test_predictions_log)
y_true = y_true_prices.values

print(f"Initial OOF SMAPE from model: {smape(y_true, oof_preds):.4f}")

# --- AGGRESSIVE STEP 1: Quantile Recalibration ---
# Force the prediction distribution to exactly match the training distribution.
print("Applying Quantile Recalibration...")
oof_ranks = rankdata(oof_preds) / len(oof_preds)
oof_preds_calibrated = np.quantile(y_true, oof_ranks)
test_ranks = rankdata(test_preds) / len(test_preds)
test_preds_calibrated = np.quantile(y_true, test_ranks)
print(f"OOF SMAPE after Quantile Recalibration: {smape(y_true, oof_preds_calibrated):.4f}")

# --- AGGRESSIVE STEP 2: Extreme Brute-Force Optimization ---
# Find the absolute best scaling and shifting factor to minimize SMAPE.
print("\nStarting brute-force optimization...")
best_smape = smape(y_true, oof_preds_calibrated)
best_s = 1.0
best_sh = 0.0
y_pred_base = oof_preds_calibrated.copy()
search_space_s = np.linspace(0.90, 1.10, 41)
search_space_sh = np.linspace(-0.1, 0.1, 41)

for s in tqdm(search_space_s, desc="Optimizing"):
    for sh in search_space_sh:
        adj_preds = np.clip(y_pred_base * s + sh * y_pred_base.std(), 0.01, None)
        current_smape = smape(y_true, adj_preds)
        if current_smape < best_smape:
            best_smape = current_smape
            best_s = s
            best_sh = sh

print(f"\n--- FINAL BEST POSSIBLE OOF SMAPE: {best_smape:.4f} ---")
print(f"(Found optimal scale={best_s:.4f}, shift={best_sh:.4f})")

# --- FINAL STEP: Apply the optimal parameters and submit ---
print("\nApplying final transformation to test predictions...")
final_test_preds = test_preds_calibrated * best_s + best_sh * test_preds_calibrated.std()
final_test_preds = np.clip(final_test_preds, 0.01, None)

original_test_df = pd.read_csv("../data/raw/dataset/test.csv")
submission_df = pd.DataFrame({'sample_id': original_test_df['sample_id'], 'price': final_test_preds})
submission_df.to_csv("../test_out.csv", index=False)

print(f"\n✅✅✅ AT-ANY-COST SUBMISSION FILE CREATED: 'test_out.csv'. SUBMIT IT NOW! ✅✅✅")

--- STARTING AT-ANY-COST HYPER-AGGRESSIVE POST-PROCESSING ---
Initial OOF SMAPE from model: 54.1383
Applying Quantile Recalibration...
OOF SMAPE after Quantile Recalibration: 60.6149

Starting brute-force optimization...


Optimizing: 100%|██████████| 41/41 [00:00<00:00, 52.69it/s]



--- FINAL BEST POSSIBLE OOF SMAPE: 57.0947 ---
(Found optimal scale=0.9000, shift=0.0550)

Applying final transformation to test predictions...

✅✅✅ AT-ANY-COST SUBMISSION FILE CREATED: 'test_out.csv'. SUBMIT IT NOW! ✅✅✅


In [5]:
import numpy as np
import pandas as pd
from scipy.stats import rankdata
from tqdm import tqdm

print("--- STARTING FINAL, AT-ANY-COST POST-PROCESSING ---")

# --- We will use the variables from the previous cell's run ---
# y_true_prices, oof_predictions_log, test_predictions_log, X, X_test

# --- Define the canonical SMAPE function ---
def smape(y_true, y_pred):
    denom = np.abs(y_true) + np.abs(y_pred)
    return 100.0 * np.mean(2 * np.abs(y_true - y_pred) / np.clip(denom, 1e-8, None))

# --- Invert from log scale ---
oof_preds = np.expm1(oof_predictions_log)
test_preds = np.expm1(test_predictions_log)
y_true = y_true_prices.values

print(f"Initial OOF SMAPE from model: {smape(y_true, oof_preds):.4f}")

# --- AGGRESSIVE STEP 1: Quantile Recalibration ---
print("Applying Quantile Recalibration...")
oof_ranks = rankdata(oof_preds) / len(oof_preds)
oof_preds_calibrated = np.quantile(y_true, oof_ranks)
test_ranks = rankdata(test_preds) / len(test_preds)
test_preds_calibrated = np.quantile(y_true, test_ranks)
print(f"OOF SMAPE after Quantile Recalibration: {smape(y_true, oof_preds_calibrated):.4f}")

# --- AGGRESSIVE STEP 2: Extreme Brute-Force Optimization ---
print("\nStarting brute-force optimization...")
best_smape = smape(y_true, oof_preds_calibrated)
best_s = 1.0
best_sh = 0.0
y_pred_base = oof_preds_calibrated.copy()
# Wide and aggressive search space
search_space_s = np.linspace(0.85, 1.15, 61) 
search_space_sh = np.linspace(-0.2, 0.2, 81)

for s in tqdm(search_space_s, desc="Optimizing"):
    for sh in search_space_sh:
        adj_preds = np.clip(y_pred_base * s + sh * y_pred_base.std(), 0.01, None)
        current_smape = smape(y_true, adj_preds)
        if current_smape < best_smape:
            best_smape = current_smape
            best_s = s
            best_sh = sh

print(f"\n--- FINAL BEST POSSIBLE OOF SMAPE: {best_smape:.4f} ---")
print(f"(Found optimal scale={best_s:.4f}, shift={best_sh:.4f})")

# --- FINAL STEP: Apply the optimal parameters and submit ---
print("\nApplying final transformation to test predictions...")
final_test_preds = test_preds_calibrated * best_s + best_sh * test_preds_calibrated.std()
final_test_preds = np.clip(final_test_preds, 0.01, None)

original_test_df = pd.read_csv("../data/raw/dataset/test.csv")
submission_df = pd.DataFrame({'sample_id': original_test_df['sample_id'], 'price': final_test_preds})
submission_df.to_csv("../test_out.csv", index=False)

print(f"\n✅✅✅ FINAL AT-ANY-COST SUBMISSION FILE CREATED: 'test_out.csv'. SUBMIT IT NOW! ✅✅✅")

--- STARTING FINAL, AT-ANY-COST POST-PROCESSING ---
Initial OOF SMAPE from model: 54.1383
Applying Quantile Recalibration...
OOF SMAPE after Quantile Recalibration: 60.6149

Starting brute-force optimization...


Optimizing: 100%|██████████| 61/61 [00:02<00:00, 20.73it/s]



--- FINAL BEST POSSIBLE OOF SMAPE: 56.4572 ---
(Found optimal scale=0.8500, shift=0.0650)

Applying final transformation to test predictions...

✅✅✅ FINAL AT-ANY-COST SUBMISSION FILE CREATED: 'test_out.csv'. SUBMIT IT NOW! ✅✅✅


In [7]:
import numpy as np
import pandas as pd
from scipy.stats import rankdata
from tqdm import tqdm

print("--- STARTING NUCLEAR OPTION: FINAL AGGRESSIVE POST-PROCESSING ---")

# --- Use variables from the previous cell's run ---
# y_true_prices, oof_predictions_log, test_predictions_log

# --- Define SMAPE ---
def smape(y_true, y_pred):
    denom = np.abs(y_true) + np.abs(y_pred)
    return 100.0 * np.mean(2 * np.abs(y_true - y_pred) / np.clip(denom, 1e-8, None))

# --- Invert from log scale ---
oof_preds = np.expm1(oof_predictions_log)
test_preds = np.expm1(test_predictions_log)
y_true = y_true_prices.values
print(f"Initial OOF SMAPE: {smape(y_true, oof_preds):.4f}")

# --- GAUNTLET STEP 1: Quantile Recalibration (Most Powerful) ---
print("Applying Quantile Recalibration...")
oof_ranks = rankdata(oof_preds) / len(oof_preds)
oof_preds_calibrated = np.quantile(y_true, oof_ranks)
test_ranks = rankdata(test_preds) / len(test_preds)
test_preds_calibrated = np.quantile(y_true, test_ranks)
print(f"After Quantile Recalibration: {smape(y_true, oof_preds_calibrated):.4f}")

# --- GAUNTLET STEP 2: Brute-Force Optimization ---
print("\nStarting brute-force scale/shift optimization...")
best_smape = smape(y_true, oof_preds_calibrated)
best_s, best_sh = 1.0, 0.0
y_pred_base = oof_preds_calibrated.copy()
for s in np.linspace(0.90, 1.10, 41):
    for sh in np.linspace(-0.1, 0.1, 41):
        adj_preds = np.clip(y_pred_base * s + sh * y_pred_base.std(), 0.01, None)
        current_smape = smape(y_true, adj_preds)
        if current_smape < best_smape:
            best_smape = current_smape; best_s = s; best_sh = sh
print(f"After Brute-Force Optimization: {best_smape:.4f} (s={best_s:.2f}, sh={best_sh:.2f})")

# Apply the best scale/shift found
oof_preds_optimized = oof_preds_calibrated * best_s + best_sh * oof_preds_calibrated.std()
test_preds_optimized = test_preds_calibrated * best_s + best_sh * test_preds_calibrated.std()

# --- GAUNTLET STEP 3: Final Median Smoothing Blend (Last Resort Trick) ---
print("\nApplying final median smoothing blend...")
best_blend_smape = smape(y_true, oof_preds_optimized)
best_alpha = 1.0
global_median = np.median(y_true)
for alpha in np.linspace(0.85, 1.0, 31):
    blend_preds = alpha * oof_preds_optimized + (1 - alpha) * global_median
    current_smape = smape(y_true, blend_preds)
    if current_smape < best_blend_smape:
        best_blend_smape = current_smape
        best_alpha = alpha

print(f"--- ABSOLUTE FINAL BEST OOF SMAPE: {best_blend_smape:.4f} --- (alpha={best_alpha:.2f})")

# --- FINAL SUBMISSION ---
print("\nApplying ALL transformations to test predictions...")
final_test_preds = test_preds_optimized * best_alpha + (1 - best_alpha) * global_median
final_test_preds = np.clip(final_test_preds, 0.01, None)

original_test_df = pd.read_csv("../data/raw/dataset/test.csv")
submission_df = pd.DataFrame({'sample_id': original_test_df['sample_id'], 'price': final_test_preds})
submission_df.to_csv("../test_out.csv", index=False)
print(f"\n✅✅✅ NUCLEAR OPTION SUBMISSION FILE CREATED: 'test_out.csv'. SUBMIT IT NOW! ✅✅✅")

--- STARTING NUCLEAR OPTION: FINAL AGGRESSIVE POST-PROCESSING ---
Initial OOF SMAPE: 54.1383
Applying Quantile Recalibration...
After Quantile Recalibration: 60.6149

Starting brute-force scale/shift optimization...
After Brute-Force Optimization: 57.0947 (s=0.90, sh=0.05)

Applying final median smoothing blend...
--- ABSOLUTE FINAL BEST OOF SMAPE: 56.0938 --- (alpha=0.85)

Applying ALL transformations to test predictions...

✅✅✅ NUCLEAR OPTION SUBMISSION FILE CREATED: 'test_out.csv'. SUBMIT IT NOW! ✅✅✅


In [8]:
import numpy as np
import pandas as pd
from scipy.stats import rankdata
from tqdm import tqdm

print("--- STARTING AT-ANY-COST HYPER-AGGRESSIVE POST-PROCESSING ---")

# --- We will use the variables from the previous cell's run:
# y_true_prices, oof_predictions_log, test_predictions_log

# --- Define the canonical SMAPE function ---
def smape(y_true, y_pred):
    denom = np.abs(y_true) + np.abs(y_pred)
    return 100.0 * np.mean(2 * np.abs(y_true - y_pred) / np.clip(denom, 1e-8, None))

# --- Invert from log scale ---
oof_preds = np.expm1(oof_predictions_log)
test_preds = np.expm1(test_predictions_log)
y_true = y_true_prices.values

print(f"Initial OOF SMAPE from model: {smape(y_true, oof_preds):.4f}")

# --- AGGRESSIVE STEP 1: Quantile Recalibration (Your Most Aggressive Idea ⚔️ 2) ---
# This forces the prediction distribution to exactly match the training distribution.
print("Applying Quantile Recalibration...")
# Create ranks from the OOF predictions
oof_ranks = rankdata(oof_preds) / len(oof_preds)
# Find the corresponding quantiles from the true training data
oof_preds_calibrated = np.quantile(y_true, oof_ranks)

# Apply the same logic to the test predictions
test_ranks = rankdata(test_preds) / len(test_preds)
test_preds_calibrated = np.quantile(y_true, test_ranks) # Use y_true as the reference distribution
print(f"OOF SMAPE after Quantile Recalibration: {smape(y_true, oof_preds_calibrated):.4f}")


# --- AGGRESSIVE STEP 2: Extreme Brute-Force Optimization (Your Idea ⚔️ 1) ---
# We search for the best possible scaling and shifting factor on the calibrated OOF preds.
print("\nStarting brute-force optimization...")
best_smape = smape(y_true, oof_preds_calibrated)
best_s = 1.0
best_sh = 0.0
y_pred_base = oof_preds_calibrated.copy()
# Search a wide range of scale and shift factors
search_space_s = np.linspace(0.85, 1.15, 61) 
search_space_sh = np.linspace(-0.2, 0.2, 81)

for s in tqdm(search_space_s, desc="Optimizing"):
    for sh in search_space_sh:
        # Apply shift relative to the standard deviation
        adj_preds = y_pred_base * s + sh * y_pred_base.std()
        
        # Always clip after adjustment
        adj_preds = np.clip(adj_preds, 0.01, None)
        
        current_smape = smape(y_true, adj_preds)
        
        if current_smape < best_smape:
            best_smape = current_smape
            best_s = s
            best_sh = sh

print(f"\n--- FINAL BEST POSSIBLE OOF SMAPE: {best_smape:.4f} ---")
print(f"(Found optimal scale={best_s:.4f}, shift={best_sh:.4f})")


# --- FINAL STEP: Apply the optimal parameters and submit ---
print("\nApplying final transformation to test predictions...")
final_test_preds = test_preds_calibrated * best_s + best_sh * test_preds_calibrated.std()
final_test_preds = np.clip(final_test_preds, 0.01, None) # Final safety clip

# Create Submission File
original_test_df = pd.read_csv("../data/raw/dataset/test.csv")
submission_df = pd.DataFrame({'sample_id': original_test_df['sample_id'], 'price': final_test_preds})
submission_df.to_csv("../test_out.csv", index=False)

print(f"\n✅✅✅ AT-ANY-COST SUBMISSION FILE CREATED: 'test_out.csv'. SUBMIT IT NOW! ✅✅✅")

--- STARTING AT-ANY-COST HYPER-AGGRESSIVE POST-PROCESSING ---
Initial OOF SMAPE from model: 54.1383
Applying Quantile Recalibration...
OOF SMAPE after Quantile Recalibration: 60.6149

Starting brute-force optimization...


Optimizing: 100%|██████████| 61/61 [00:03<00:00, 17.24it/s]



--- FINAL BEST POSSIBLE OOF SMAPE: 56.4572 ---
(Found optimal scale=0.8500, shift=0.0650)

Applying final transformation to test predictions...

✅✅✅ AT-ANY-COST SUBMISSION FILE CREATED: 'test_out.csv'. SUBMIT IT NOW! ✅✅✅


In [9]:
import numpy as np
import pandas as pd
from scipy.stats import rankdata
from tqdm import tqdm

print("--- STARTING NUCLEAR OPTION: THE FINAL POST-PROCESSING SCRIPT ---")

# --- Use variables from the previous cell's run ---
# y_true_prices, oof_predictions_log, test_predictions_log

# --- Define SMAPE ---
def smape(y_true, y_pred):
    denom = np.abs(y_true) + np.abs(y_pred)
    return 100.0 * np.mean(2 * np.abs(y_true - y_pred) / np.clip(denom, 1e-8, None))

# --- Invert from log scale ---
oof_preds = np.expm1(oof_predictions_log)
test_preds = np.expm1(test_predictions_log)
y_true = y_true_prices.values
print(f"Initial OOF SMAPE: {smape(y_true, oof_preds):.4f}")

# --- AGGRESSIVE STEP 1: Quantile Recalibration (The most powerful distribution-matching tool) ---
print("Applying Quantile Recalibration...")
oof_ranks = rankdata(oof_preds) / len(oof_preds)
oof_preds_calibrated = np.quantile(y_true, oof_ranks)
test_ranks = rankdata(test_preds) / len(test_preds)
test_preds_calibrated = np.quantile(y_true, test_ranks)
print(f"OOF SMAPE after Quantile Recalibration: {smape(y_true, oof_preds_calibrated):.4f}")

# --- AGGRESSIVE STEP 2: Extreme Blending with Global Median ---
# This is a high-risk move to aggressively pull all predictions towards a safe central value.
print("\nStarting aggressive blend optimization...")
best_smape = smape(y_true, oof_preds_calibrated)
best_alpha = 1.0
global_median = np.median(y_true)

# Search for the perfect blend between our calibrated predictions and the safe global median
for alpha in tqdm(np.linspace(0.5, 1.0, 101), desc="Finding Best Blend"):
    blend_preds = alpha * oof_preds_calibrated + (1 - alpha) * global_median
    current_smape = smape(y_true, blend_preds)
    if current_smape < best_smape:
        best_smape = current_smape
        best_alpha = alpha

print(f"\n--- ABSOLUTE FINAL BEST OOF SMAPE: {best_smape:.4f} ---")
print(f"(Found optimal blend alpha={best_alpha:.2f})")

# --- FINAL SUBMISSION ---
print("\nApplying ALL transformations to test predictions...")
final_test_preds = best_alpha * test_preds_calibrated + (1 - best_alpha) * global_median
final_test_preds = np.clip(final_test_preds, 0.01, None)

original_test_df = pd.read_csv("../data/raw/dataset/test.csv")
submission_df = pd.DataFrame({'sample_id': original_test_df['sample_id'], 'price': final_test_preds})
submission_df.to_csv("../test_out.csv", index=False)

print(f"\n✅✅✅✅✅ NUCLEAR OPTION SUBMISSION FILE CREATED: 'test_out.csv'. SUBMIT IT NOW! ✅✅✅✅✅")

--- STARTING NUCLEAR OPTION: THE FINAL POST-PROCESSING SCRIPT ---
Initial OOF SMAPE: 54.1383
Applying Quantile Recalibration...
OOF SMAPE after Quantile Recalibration: 60.6149

Starting aggressive blend optimization...


Finding Best Blend: 100%|██████████| 101/101 [00:00<00:00, 1567.93it/s]


--- ABSOLUTE FINAL BEST OOF SMAPE: 55.6435 ---
(Found optimal blend alpha=0.70)

Applying ALL transformations to test predictions...



✅✅✅✅✅ NUCLEAR OPTION SUBMISSION FILE CREATED: 'test_out.csv'. SUBMIT IT NOW! ✅✅✅✅✅


In [10]:
import numpy as np
import pandas as pd
from scipy.stats import rankdata
from tqdm import tqdm

print("--- FINAL ATTEMPT: AGGRESSIVE RECALIBRATION ---")

# --- Use variables from the previous cell's run ---
# y_true_prices, oof_predictions_log, test_predictions_log

# --- Define SMAPE ---
def smape(y_true, y_pred):
    denom = np.abs(y_true) + np.abs(y_pred)
    return 100.0 * np.mean(2 * np.abs(y_true - y_pred) / np.clip(denom, 1e-8, None))

# --- Invert from log scale ---
oof_preds = np.expm1(oof_predictions_log)
test_preds = np.expm1(test_predictions_log)
y_true = y_true_prices.values

print(f"Initial OOF SMAPE: {smape(y_true, oof_preds):.4f}")

# --- THE MOST POWERFUL TRICK: Quantile Recalibration ---
# This forces the prediction distribution to exactly match the training distribution.
# This is our best and only shot at a massive score drop.
print("Applying Quantile Recalibration...")
# Create ranks from the OOF predictions
oof_ranks = rankdata(oof_preds) / len(oof_preds)
# Find the corresponding quantiles from the true training data
oof_preds_calibrated = np.quantile(y_true, oof_ranks)

# Apply the same logic to the test predictions
test_ranks = rankdata(test_preds) / len(test_preds)
final_test_preds = np.quantile(y_true, test_ranks) # Use y_true as the reference distribution

print(f"--- FINAL OOF SMAPE AFTER RECALIBRATION: {smape(y_true, oof_preds_calibrated):.4f} ---")


# --- Final safety clip and submit ---
final_test_preds = np.clip(final_test_preds, 0.01, None)

original_test_df = pd.read_csv("../data/raw/dataset/test.csv")
submission_df = pd.DataFrame({'sample_id': original_test_df['sample_id'], 'price': final_test_preds})
submission_df.to_csv("../test_out.csv", index=False)

print(f"\n✅✅✅ FINAL GAMBLE SUBMISSION FILE CREATED: 'test_out.csv'. SUBMIT IT NOW! ✅✅✅")

--- FINAL ATTEMPT: AGGRESSIVE RECALIBRATION ---
Initial OOF SMAPE: 54.1383
Applying Quantile Recalibration...
--- FINAL OOF SMAPE AFTER RECALIBRATION: 60.6149 ---

✅✅✅ FINAL GAMBLE SUBMISSION FILE CREATED: 'test_out.csv'. SUBMIT IT NOW! ✅✅✅


In [11]:
import numpy as np
import pandas as pd
from scipy.stats import rankdata
from tqdm import tqdm

print("--- FINAL ATTEMPT: AGGRESSIVE RECALIBRATION ---")

# --- Use variables from the previous cell's run ---
# y_true_prices, oof_predictions_log, test_predictions_log

# --- Define SMAPE ---
def smape(y_true, y_pred):
    denom = np.abs(y_true) + np.abs(y_pred)
    return 100.0 * np.mean(2 * np.abs(y_true - y_pred) / np.clip(denom, 1e-8, None))

# --- Invert from log scale ---
oof_preds = np.expm1(oof_predictions_log)
test_preds = np.expm1(test_predictions_log)
y_true = y_true_prices.values

print(f"Initial OOF SMAPE: {smape(y_true, oof_preds):.4f}")

# --- THE MOST POWERFUL TRICK: Quantile Recalibration ---
# This forces the prediction distribution to exactly match the training distribution.
# This is our best and only shot at a massive score drop.
print("Applying Quantile Recalibration...")
# Create ranks from the OOF predictions
oof_ranks = rankdata(oof_preds) / len(oof_preds)
# Find the corresponding quantiles from the true training data
oof_preds_calibrated = np.quantile(y_true, oof_ranks)

# Apply the same logic to the test predictions
test_ranks = rankdata(test_preds) / len(test_preds)
final_test_preds = np.quantile(y_true, test_ranks) # Use y_true as the reference distribution

print(f"--- FINAL OOF SMAPE AFTER RECALIBRATION: {smape(y_true, oof_preds_calibrated):.4f} ---")


# --- Final safety clip and submit ---
final_test_preds = np.clip(final_test_preds, 0.01, None)

original_test_df = pd.read_csv("../data/raw/dataset/test.csv")
submission_df = pd.DataFrame({'sample_id': original_test_df['sample_id'], 'price': final_test_preds})
submission_df.to_csv("../test_out.csv", index=False)

print(f"\n✅✅✅ FINAL GAMBLE SUBMISSION FILE CREATED: 'test_out.csv'. SUBMIT IT NOW! ✅✅✅")

--- FINAL ATTEMPT: AGGRESSIVE RECALIBRATION ---
Initial OOF SMAPE: 54.1383
Applying Quantile Recalibration...
--- FINAL OOF SMAPE AFTER RECALIBRATION: 60.6149 ---

✅✅✅ FINAL GAMBLE SUBMISSION FILE CREATED: 'test_out.csv'. SUBMIT IT NOW! ✅✅✅


In [13]:
import numpy as np
import pandas as pd

print("--- CREATING THE SAFEST, MOST RELIABLE SUBMISSION ---")

# --- Use the original, pure model predictions from memory ---
# oof_predictions_log, test_predictions_log, y_true_prices

# --- Step 1: Invert from log scale ---
test_preds = np.expm1(test_predictions_log)

# --- Step 2: Apply ONLY the safest, most essential clipping ---
# Clip at the bottom to prevent zero/negative prices.
# Clip at the top to prevent prices higher than anything in the training set.
final_test_preds = np.clip(test_preds, 0.01, y_true_prices.max())

print(f"Final predictions created. Range: Min={final_test_preds.min():.2f}, Max={final_test_preds.max():.2f}")

# --- Create the Submission File ---
original_test_df = pd.read_csv("../data/raw/dataset/test.csv")
submission_df = pd.DataFrame({'sample_id': original_test_df['sample_id'], 'price': final_test_preds})
submission_df.to_csv("../test_out.csv", index=False)

print(f"\n✅✅✅ SAFEST FINAL SUBMISSION CREATED: 'test_out.csv'. SUBMIT THIS NOW! ✅✅✅")

--- CREATING THE SAFEST, MOST RELIABLE SUBMISSION ---
Final predictions created. Range: Min=0.84, Max=254.25

✅✅✅ SAFEST FINAL SUBMISSION CREATED: 'test_out.csv'. SUBMIT THIS NOW! ✅✅✅


In [14]:
import numpy as np
import pandas as pd
from tqdm import tqdm

print("--- STARTING FINAL 'REDUCE 20' SCRIPT ---")

# --- Use variables from the previous cell's run ---
# y_true_prices, oof_predictions_log, test_predictions_log, X, X_test

# --- Define SMAPE ---
def smape(y_true, y_pred):
    denom = np.abs(y_true) + np.abs(y_pred)
    return 100.0 * np.mean(2 * np.abs(y_true - y_pred) / np.clip(denom, 1e-8, None))

# --- Invert from log scale ---
oof_preds = np.expm1(oof_predictions_log)
test_preds = np.expm1(test_predictions_log)
y_true = y_true_prices.values
print(f"Initial OOF SMAPE: {smape(y_true, oof_preds):.4f}")

# --- GAUNTLET STEP 1: Winsorize (Clip extreme tails) ---
low, high = np.percentile(y_true, [1, 99]) # Aggressive clipping
oof_preds = np.clip(oof_preds, low, high)
test_preds = np.clip(test_preds, low, high)
print(f"After Winsorizing: {smape(y_true, oof_preds):.4f}")

# --- GAUNTLET STEP 2: Group-wise Mean Correction ---
train_brands = X['brand']
test_brands = X_test['brand']
oof_df = pd.DataFrame({'brand': train_brands, 'y_true': y_true, 'pred': oof_preds})
group_shift = oof_df.groupby('brand').apply(lambda g: g['y_true'].mean() - g['pred'].mean()).to_dict()
oof_preds += train_brands.map(group_shift).fillna(0.0).values
test_preds += test_brands.map(group_shift).fillna(0.0).values
print(f"After Group Correction: {smape(y_true, oof_preds):.4f}")

# --- GAUNTLET STEP 3: Brute-Force Optimization ---
print("\nStarting brute-force optimization...")
best_smape = smape(y_true, oof_preds)
best_s, best_sh = 1.0, 0.0
y_pred_base = oof_preds.copy()
# Very wide and aggressive search
search_space_s = np.linspace(0.8, 1.2, 81) 
search_space_sh = np.linspace(-0.25, 0.25, 101)

for s in tqdm(search_space_s, desc="Optimizing"):
    for sh in search_space_sh:
        adj_preds = np.clip(y_pred_base * s + sh * y_pred_base.std(), 0.01, None)
        current_smape = smape(y_true, adj_preds)
        if current_smape < best_smape:
            best_smape = current_smape; best_s = s; best_sh = sh

print(f"\n--- FINAL BEST POSSIBLE OOF SMAPE: {best_smape:.4f} ---")
print(f"(Found optimal scale={best_s:.4f}, shift={best_sh:.4f})")

# --- FINAL SUBMISSION ---
print("\nApplying ALL transformations to test predictions...")
final_test_preds = test_preds * best_s + best_sh * test_preds.std()
final_test_preds = np.clip(final_test_preds, 0.01, None)

original_test_df = pd.read_csv("../data/raw/dataset/test.csv")
submission_df = pd.DataFrame({'sample_id': original_test_df['sample_id'], 'price': final_test_preds})
submission_df.to_csv("../test_out.csv", index=False)

print(f"\n✅✅✅✅✅ FINAL GAMBLE SUBMISSION CREATED: 'test_out.csv'. SUBMIT IT NOW! ✅✅✅✅✅")

--- STARTING FINAL 'REDUCE 20' SCRIPT ---
Initial OOF SMAPE: 54.1383
After Winsorizing: 54.1362
After Group Correction: 22.9111

Starting brute-force optimization...


Optimizing: 100%|██████████| 81/81 [00:05<00:00, 15.69it/s]



--- FINAL BEST POSSIBLE OOF SMAPE: 22.9069 ---
(Found optimal scale=1.0000, shift=0.0000)

Applying ALL transformations to test predictions...

✅✅✅✅✅ FINAL GAMBLE SUBMISSION CREATED: 'test_out.csv'. SUBMIT IT NOW! ✅✅✅✅✅


In [15]:
import numpy as np
import pandas as pd

print("--- CREATING THE FINAL, BALANCED AND SAFEST SUBMISSION ---")

# --- Use the original, pure model predictions from memory ---
# oof_predictions_log, test_predictions_log, y_true_prices, X, X_test

# --- Define SMAPE ---
def smape(y_true, y_pred):
    denom = np.abs(y_true) + np.abs(y_pred)
    return 100.0 * np.mean(2 * np.abs(y_true - y_pred) / np.clip(denom, 1e-8, None))

# --- Step 1: Invert from log scale ---
oof_preds = np.expm1(oof_predictions_log)
test_preds = np.expm1(test_predictions_log)
y_true = y_true_prices.values

print(f"Initial (pure model) OOF SMAPE: {smape(y_true, oof_preds):.4f}")

# --- Step 2: Apply ONLY the most reliable post-processing: Group-wise Mean Correction ---
# This is the most powerful trick that is least likely to overfit.
print("Applying ONLY Group-wise Mean Correction...")
train_brands = X['brand']
test_brands = X_test['brand']
oof_df = pd.DataFrame({'brand': train_brands, 'y_true': y_true, 'pred': oof_preds})
group_shift = oof_df.groupby('brand').apply(lambda g: g['y_true'].mean() - g['pred'].mean()).to_dict()

oof_preds_final = oof_preds + train_brands.map(group_shift).fillna(0.0).values
final_test_preds = test_preds + test_brands.map(group_shift).fillna(0.0).values
print(f"OOF SMAPE after Group Correction: {smape(y_true, oof_preds_final):.4f}")


# --- Step 3: Final Safety Clip ---
final_test_preds = np.clip(final_test_preds, 0.01, y_true.max())
print(f"\nFinal predictions created. Range: Min={final_test_preds.min():.2f}, Max={final_test_preds.max():.2f}")

# --- Create the Submission File ---
original_test_df = pd.read_csv("../data/raw/dataset/test.csv")
submission_df = pd.DataFrame({'sample_id': original_test_df['sample_id'], 'price': final_test_preds})
submission_df.to_csv("../test_out.csv", index=False)

print(f"\n✅✅✅ FINAL BALANCED SUBMISSION CREATED: 'test_out.csv'. SUBMIT THIS NOW! ✅✅✅")

--- CREATING THE FINAL, BALANCED AND SAFEST SUBMISSION ---
Initial (pure model) OOF SMAPE: 54.1383
Applying ONLY Group-wise Mean Correction...
OOF SMAPE after Group Correction: 22.9081

Final predictions created. Range: Min=0.01, Max=683.45

✅✅✅ FINAL BALANCED SUBMISSION CREATED: 'test_out.csv'. SUBMIT THIS NOW! ✅✅✅
